In [1]:
from services.load_food_embedding import df_recipes, food_embeddings, food_names, model

Embeddings loaded successfully!


In [2]:
from langchain_core.tools import tool

In [3]:
@tool
def find_best_match(query:str)->str:
    """Find the best match for a given query in the food names.
    Args:
        query (str): The input query to find the best match for.
    Returns:
        str: The food name that best matches the query.
    """
    # Ensure the query is a string
    if not isinstance(query, str):
        raise ValueError("Query must be a string")
    query_embedding = model.encode([query])
    similarities = cosine_similarity(query_embedding, food_embeddings)
    best_idx = np.argmax(similarities)
    return food_names[best_idx]
@tool
def get_food_details(food_name:str)->dict:
    """Get food details from the DataFrame.
    Args:
        food_name (str): The name of the food to retrieve details for.
    Returns:
        dict: A dictionary containing the food details, or None if not found.
    """
    food_details = df_recipes[df_recipes["name"] == food_name]
    if not food_details.empty:
        return food_details.to_dict(orient="records")[0]
    else:
        return None
@tool
def find_best_food_match(text_array:list[str])->str:
    """Extract the most relevant food name by finding the closest match in recipe embeddings.
    Args:
        text_array (list[str]): A list of strings to find the best food match for.
    Returns:
        str: The food name that best matches the input text array.
    Raises:
        ValueError: If the input is not a list of strings.
    """
    # Convert input list into a single meaningful string
    combined_text = " ".join(text_array)
    
    # Generate embedding for input text
    input_embedding = model.encode([combined_text], convert_to_numpy=True)

    # Compute cosine similarity with all recipes
    similarities = cosine_similarity(input_embedding, food_embeddings)[0]

    # Get the closest match index
    best_match_idx = np.argmax(similarities)
    
    # Access food name directly if food_names is a list
    return food_names[best_match_idx]  # Replace .iloc with direct indexing for lists



In [4]:
from langchain_ollama import ChatOllama

local_llm2 = "llama3.1:8b"
llm2 = ChatOllama(model=local_llm2, temperature=0)

In [5]:
tools = [find_best_match, get_food_details, find_best_food_match]
llm_with_tools = llm2.bind_tools(tools)

In [6]:
from langchain_ollama import ChatOllama

local_llm = "granite3.2-vision:2b"
llm = ChatOllama(model=local_llm, temperature=0)

In [7]:
tools = [find_best_match, get_food_details, find_best_food_match]
llm_with_tools2 = llm.bind_tools(tools)

In [8]:
query="what is the food details of samosa"
llm_with_tools.invoke(query).tool_calls

[{'name': 'get_food_details',
  'args': {'food_name': 'samosa'},
  'id': 'a59af5b9-1697-4897-be34-59cb09aac5a9',
  'type': 'tool_call'}]

In [10]:
async for chunk in llm_with_tools.astream(query):
    print(chunk.tool_call_chunks)

[{'name': 'get_food_details', 'args': '{"food_name": "samosa"}', 'id': '1c8c0065-85c6-48d9-891a-5f2fbc08a0bb', 'index': None, 'type': 'tool_call_chunk'}]
[]
